# Introduction

Experimentation on the [UC2018 DualMyo Hand Gesture Dataset](https://zenodo.org/record/1320922#.Xi9l5xfgoWo).

In [ ]:
import os, sys, requests, pickle
import numpy as np, matplotlib.pyplot as plt
from sequentia import *
from tqdm.auto import tqdm

# Silence TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import utility functions and classes
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils import *
from lstm import LSTMClassifier

# ggplot style
plt.style.use('ggplot')

# Set seed for reproducible randomness
seed = 0
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [ ]:
n_classes = 8
classes = [str(i) for i in range(n_classes)]
classes

In [ ]:
clfs, results = {}, {'hmm': {}, 'knn': {}, 'lstm': {}}

In [ ]:
X, y = [], []
url = 'https://zenodo.org/record/1320922/files/dualmyo_dataset.pkl'

try:
    file = 'dualmyo_dataset.pkl'
    print('Downloading dataset from {} ...'.format(url))
    response = requests.get(url)
    with open(file, 'wb') as f:
        print('Writing {} ...'.format(file))
        f.write(response.content)
except:
    raise
else:
    print('Unpickling data into Numpy arrays ...')
    data = pickle.load(open(file, 'rb'))
    X = data[0]
    y = [str(label) for label in data[1]]
    print('Done!')
finally:
    os.remove(file)

## Dataset splits

Create a stratified 65-20-15 training, validation and test set split.

In [ ]:
# Create a stratified training, validation and test set split (65-20-15)
X_train, X_val, X_test, y_train, y_val, y_test = data_split(X, y, random_state=rng, stratify=True)

In [ ]:
# DualMyo dataset class counts (training set)
show_class_counts(y_train, classes, title=None)

In [ ]:
# Histogram of DualMyo dataset gesture durations (training set)
show_durations(X_train, bins=75, title=None)

## Preprocessing

In [ ]:
# Create a preprocessing pipeline
pre = Preprocess([
    Filter(window_size=5, method='median'),
    Downsample(factor=10, method='decimate'),
    Center()
])
pre.summary()

In [ ]:
# Pick an example signal for visualization
x = X_train[1]

plt.figure(figsize=(16, 3))
plt.title('Example signal before preprocessing')
plt.plot(x)
plt.show()

plt.figure(figsize=(16, 3))
plt.title('Example signal after preprocessing')
plt.plot(pre.transform(x))
plt.show()

In [ ]:
# Transform training data and plot histogram of DualMyo dataset gesture durations (preprocessed) (training set)
Xp_train = pre.transform(X_train, verbose=True)
show_durations(Xp_train, bins=75, title=None)

In [ ]:
# Apply the preprocessing pipeline to the other dataset splits
Xp_val, Xp_test = pre.transform(X_val, verbose=True), pre.transform(X_test, verbose=True)

## DTWKNN classifier

### Fitting the model

In [ ]:
%%time
# Create and fit a DTWKNN classifier using the single nearest neighbor and a radius of 1
# NOTE: The radius parameter is a parameter that constrains the FastDTW algorithm.
clfs['knn'] = DTWKNN(k=1, radius=1)
clfs['knn'].fit(Xp_train, y_train)

### Evaluating the model

In [ ]:
%%time
results['knn']['validation'] = clfs['knn'].evaluate(Xp_val, y_val, labels=classes, n_jobs=-1)
show_results(*results['knn']['validation'], dataset='validation', labels=classes)

## Hidden Markov Model classifier

### Fitting the model

In [ ]:
%%time

# Create HMMs to represent each class
#
# NumPy sometimes raises some errors as a result of instability during the Cholesky decomposition.
# According to issue #414 on Pomegranate's GitHub repository, this may be caused by:
# - Too many states in the HMMs
# - Too many dimensions in the input data, which leads to a large covariance matrix
# - Too few training examples
hmms = []
for c in tqdm(classes, desc='Training HMMs'):
    hmm = HMM(label=c, n_states=7, random_state=rng)
    hmm.set_random_initial()
    hmm.set_random_transitions()
    hmm.fit([Xp_train[i] for i, label in enumerate(y_train) if label == c])
    hmms.append(hmm)
    
# Fit a HMM classifier with the HMMs
clfs['hmm'] = HMMClassifier()
clfs['hmm'].fit(hmms)

### Evaluating the model

In [ ]:
%%time
results['hmm']['validation'] = clfs['hmm'].evaluate(Xp_val, y_val, labels=classes)
show_results(*results['hmm']['validation'], dataset='validation', labels=classes)

## LSTM classifier

### Fitting the model

In [ ]:
%%time
clfs['lstm'] = LSTMClassifier(epochs=30, batch_size=64, classes=classes)
hist = clfs['lstm'].fit(Xp_train, y_train, validation_data=(Xp_val, y_val), return_history=True)

In [ ]:
# Display accuracy history during training
show_accuracy_history(hist)

In [ ]:
# Display loss history during training
show_loss_history(hist)

### Evaluating the model

In [ ]:
%%time
results['lstm']['validation'] = clfs['lstm'].evaluate(Xp_val, y_val)
show_results(*results['lstm']['validation'], dataset='validation', labels=classes)